<a href="https://colab.research.google.com/github/SinghAbh1shek/GenAI-project/blob/main/Trending_news_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Environment Variables**



> This code gets our Gemini, Newsapi and Pexels API keys from Colab secrets.

**Make sure to set your Gemini, Newsapi and Pexels API keys with the names "NEWS_API", "GOOGLE_API_KEY" and "PEXELS_API_KEY"**



In [236]:
from google.colab import userdata
GEMINI_API_KEY = userdata.get('GOOGLE_API_KEY')
PEXELS_API_KEY = userdata.get('PEXELS_API_KEY')
NEWS_API = userdata.get('NEWS_API')

#**Fetching trending News Headlines**
> This code imports the requests library, sends a request to the NewsAPI to get trending headlines using an API key, and prints the news data in JSON format.

In [237]:
import requests
url = (f"https://newsapi.org/v2/top-headlines?country=us&apiKey={NEWS_API}")
response = requests.get(url)
# print(response.json())

#**get_trending_article function**


> This function retrieves the title and description of the top article from the result and returns them as a formatted string.



In [238]:
def get_trending_article():
  result  = response.json()
  trending_article_title = result['articles'][0]['title']
  trending_article_content = result['articles'][0]['description']
  return(f"{trending_article_title} {trending_article_content}")

#**OpenAI SDK**


> This code sets up an OpenAI client using a Gemini API key and a custom base URL to access the Gemini API.



In [239]:
import os
from openai import OpenAI
client = OpenAI(
    api_key=GEMINI_API_KEY,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

#**Structured Output**

> This defines a data model using Pydantic where content is a required string field, used to validate the structure of an article.



In [240]:
from pydantic import BaseModel

class article_schema(BaseModel):
  content: str

#**article_generator() function**



> The function generates a short news article using an Gemini AI model based on a trending topic.



In [241]:
def article_generator():
  system_prompt = """
  You are a helpful AI assistant who specializes in writing news articles.
  You can take input from the user and, based on that prompt, generate a news article.
  The length of the article should be between 100-130 words

  Note:
    - Analyse the user prompt carefully
    - Gives the best output according to the user prompt
    - Always gives the output in the specified format
  """
  user_prompt = get_trending_article()

  completion = client.beta.chat.completions.parse(
      model="gemini-1.5-flash",
      messages=[
          {"role": "system", "content": system_prompt},
          {"role": "user", "content": user_prompt},
      ],
      response_format=article_schema,
  )

  event = completion.choices[0].message.parsed.content
  return event

#**This code installs necessary Python libraries**

In [242]:
pip install moviepy requests pillow gtts

# **This code imports necessary libraries for creating a video with images and voiceovers.**

In [243]:
import os
import requests
from PIL import Image, ImageDraw, ImageFont
from io import BytesIO
from gtts import gTTS
from moviepy.editor import ImageClip, concatenate_videoclips, AudioFileClip

#**The code creates a folder named "temp" if it doesn't already exist.**










In [244]:
TEMP_FOLDER = "temp"
os.makedirs(TEMP_FOLDER, exist_ok=True)


#**get_images_from_pexels**


> The function fetches images from Pexels based on a query and returns a list of images. If the API call is successful, it downloads and adds the images. Otherwise, it adds blank images as a fallback.







In [245]:
def get_images_from_pexels(query, num_images=3):
    headers = {"Authorization": PEXELS_API_KEY}
    params = {"query": query, "per_page": num_images}
    response = requests.get("https://api.pexels.com/v1/search", headers=headers, params=params)
    images = []

    if response.status_code == 200 and response.json()["photos"]:
        for photo in response.json()["photos"]:
            img_url = photo["src"]["large"]
            img_data = requests.get(img_url).content
            images.append(Image.open(BytesIO(img_data)))
    else:
        # Fallback to blank image
        for _ in range(num_images):
            images.append(Image.new('RGB', (1280, 720), color=(30, 30, 30)))

    return images

#**create_image_with_text**


> The function resizes an image to specified size, overlays text onto it by splitting the text into lines that fit within the image width, and places the text at the bottom center. It then saves the modified image to the specified output path.



In [246]:
def create_image_with_text(image, text, output_path):
    image = image.resize((1280, 720))

    draw = ImageDraw.Draw(image)

    font_size = 25
    try:
        font = ImageFont.truetype("arial.ttf", font_size)
    except:
        font = ImageFont.load_default(font_size)

    # Split text into lines
    margin = 40
    max_width = image.width - 2 * margin
    lines = []
    words = text.split()
    line = ""

    for word in words:
        if draw.textbbox((0, 0), line + " " + word, font=font)[2] <= max_width:
            line += " " + word    #Words fits with the same line
        else:
            lines.append(line) # For new line
            line = word

    lines.append(line)

    y = image.height - (font_size + 10) * len(lines) - margin # Calculate position to place the text to bottom center
    for line in lines:
        width, _ = draw.textbbox((0, 0), line, font=font)[2:4] # Center the text horizontally
        x = (image.width - width) // 2
        draw.text((x, y), line, font=font, fill='white')
        y += font_size + 10  # Move to the next line

    image.save(output_path)


#**generate_voiceover function**


> The function converts the provided text into speech using Google Text-to-Speech (gTTS) and saves the audio as a file at the specified location.



In [247]:
def generate_voiceover(text, output_path):
    tts = gTTS(text)
    tts.save(output_path)
    return output_path

#**create_video_with_audio function**

> The function takes a script, turns it into a voiceover, and finds related images for each part of the script. It adds the script text to the images, puts everything together into a video, and saves it with the voiceover and images.

In [248]:
def create_video_with_audio(script_text, output_file="pexels_news_video_with_audio.mp4"):
    sentences = [s.strip() for s in script_text.split('.') if s.strip()]
    clips = []

    print("🎙️ Generating voiceover...")
    audio_path = os.path.join(TEMP_FOLDER, "voiceover.mp3")
    generate_voiceover(" ".join(sentences), audio_path)
    audio_clip = AudioFileClip(audio_path)

    total_duration = audio_clip.duration
    duration_per_sentence = total_duration / len(sentences)

    print("🖼️ Creating visuals for each sentence...")
    for i, sentence in enumerate(sentences):
        images = get_images_from_pexels(sentence, num_images=3)
        subclips = []
        for j, image in enumerate(images):
            img_path = os.path.join(TEMP_FOLDER, f"frame_{i}_{j}.jpg")
            create_image_with_text(image, sentence, img_path)
            subclip = ImageClip(img_path).set_duration(duration_per_sentence / len(images))
            subclips.append(subclip)

        sentence_clip = concatenate_videoclips(subclips)
        clips.append(sentence_clip)

    print("🎬 Finalizing video...")
    final_video = concatenate_videoclips(clips, method="compose").set_audio(audio_clip)
    final_video.write_videofile(output_file, fps=24)
    print(f"✅ Done! Video saved as: {output_file}")

#**Generate the article**


> This code generates a news article using the article_generator() function and then creates a video with audio based on the generated article using the create_video_with_audio() function.



In [249]:
script = article_generator()
create_video_with_audio(script)

🎙️ Generating voiceover...
🖼️ Creating visuals for each sentence...
🎬 Finalizing video...
Moviepy - Building video pexels_news_video_with_audio.mp4.
MoviePy - Writing audio in pexels_news_video_with_audioTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video pexels_news_video_with_audio.mp4



Moviepy - Done !
Moviepy - video ready pexels_news_video_with_audio.mp4
✅ Done! Video saved as: pexels_news_video_with_audio.mp4
